In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as pt

In [14]:
def generate_one_episode(center, max_move, p_r=0.5):
    left = 0
    right = 2*center
    state = center
    episode = []
    while (state>left) and (state<right):
        if npr.uniform()<=p_r:
            next_state = min(state+npr.choice(max_move), right)
        else:
            next_state = max(state-npr.choice(max_move), left)
        if next_state==right:
            reward = 1
        elif next_state==left:
            reward = -1
        else:
            reward = 0
        episode.append((state, reward))
        state = next_state
    return episode

In [17]:
episode = generate_one_episode(500, 100)

print(episode)

[(500, 0), (571, 0), (658, 0), (622, 0), (580, 0), (672, 0), (708, 0), (776, 0), (772, 0), (762, 0), (700, 0), (657, 0), (587, 0), (640, 0), (716, 0), (728, 0), (660, 0), (728, 0), (807, 0), (852, 0), (870, 0), (957, 1)]
